In [1]:
from autogen import ConversableAgent
from openai import OpenAI
from pathlib import Path

import LLM_configs
import Prompt_dics
from LLM_configs import *
from Agents import *
from tools import *

In [2]:
# 加载 .env 文件
load_dotenv()

True

## 文件提取

In [3]:
text = file_extraction("test files/永誉天泽7月实地调研.m4a-文稿-转写结果.docx")

In [4]:
PPT = file_extraction("test files/上海象上投资.pptx")

## Agent设计

In [5]:
quant_dict = {
    'quant_idea':[],
    'factor_composition':[],
    'factor_explanation':[],
    'model_explanation':[],
    'factor_update':[],
    'strategy_execution':[]
}

In [6]:
market_neutral = fill_placeholders(['中证500指增', '股指期货','0%'], Prompt_dics.Other_prompt_dic['中性多头/对冲端'])

In [7]:
market_neutral = ''

In [8]:
def report_writing(weak_config, strong_config,prompts,component_dict, requirements, updating):
    
    ## 定义信息抽取，写报告，修改报告的 agent
    extraction_agent = extraction_assistant(weak_config)
    
    integrate_agent = integrate_assistant(strong_config,requirements)
    
    report_modify_agent = report_modify(strong_config)
    
    ## 抽取信息
    
    for prompt, component in zip(prompts, list(component_dict.keys())):
        reply = extraction_agent.generate_reply(messages=[{"content": text, "role":"system"},
                                                      {"content": prompt, "role": "user"}])
        component_dict[component].append(reply)
    
    component_dict['hedging tools'] = market_neutral
    
    ## 根据抽取的信息撰写报告
    
    final_report = integrate_agent.generate_reply(messages=[{"content": str(component_dict), "role": "system"},
                                                                {
                                                                    "content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整，长度越长越好，但不要有过多的你自由发挥的解释，客观记录为主, 不要出现评价语言，例如“这体现了XXXX",
                                                                    "role": "user"}])
    non_updating_report = final_report
    
    
    ## 如果有旧报告，则将新报告和旧报告进行合成
    if updating:
        old_report_relevant_part = input("请把上一篇报告的配置逻辑部分复制到此处")
        final_report = integrate_agent.generate_reply(messages=[{"content": "旧报告: "+old_report_relevant_part+"新报告:"+str(final_report), "role": "system"},
                                                                {"content": "你是一个报告审查和撰写专员，现在你有一篇旧报告和一篇新报告，你要做的是在旧报告的基础上，判断新报告中的增量信息，并将新报告中的增量信息与旧报告中的现有信息相结合，完成一篇更新的报告，如果两个报告中对某一个地方有冲突，则以新报告为准。", "role": "system"},
                                                                {
                                                                    "content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整.长度越长越好，但不要有过多的你自由发挥的解释，客观记录为主, 不要出现评价语言，例如“这体现了XXXX",
                                                                    "role": "user"}])
        print(final_report)
    
    
    ## 增加一个 feedback模块
    feedback = input("请提供您的反馈，如果满意请输入'y'，如果不满意请输入您的调整建议: ")
    loop_counter = 0
    while feedback != 'y' or loop_counter>=5:
        final_report = report_modify_agent.generate_reply(messages = [{"content": "修改要求："+feedback+"，其他的一个字都不能改", "role": "system"},
                                                                      {"content": "这是报告:"+final_report, "role": "system"},
                                                                      {"content": "按照要求，对报告进行修改", "role": "user"}])
        print("生成的中间报告: ", final_report)
        
        feedback = input("请提供您的反馈，如果满意请输入'y'，如果不满意请输入您的调整建议: ")
        loop_counter += 1
            
            
    
    cost_list = [extraction_agent.get_total_usage(),integrate_agent.get_total_usage()]
    
    return final_report,non_updating_report, component_dict, cost_list

In [9]:
Ali_report,first_Ali_report, Quant_component_dic, cost_list = report_writing(LLM_configs.Qwen_weak_llm_config, LLM_configs.Qwen_strong_llm_config, Prompt_dics.Strategy_prompt_dic['Simple Quant Strategy'],quant_dict, Prompt_dics.Strategy_requirements_dic['Simple Quant Strategy'],True)


>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
管理人张润伟的量化策略综合了深度逻辑推理与先进的机器学习技术，旨在通过精密的基本面分析、量价信号的敏锐捕捉以及另类数据的独特洞察，构建出一套既稳健又灵活的投资框架。策略核心围绕着以基本面因子为主导，逐步融入并提升量价因子的有效性，特别是在国内市场环境下，量价因子的影响力显著，促使管理人在2022年10月引入了机器学习模型以进一步优化策略表现。截至2024年一季度，策略配置逻辑调整为基本面因子占比35%，人工挖掘的量价因子与机器学习量价因子分别占比35%和20%，另类因子占10%，体现出对多元化因子来源的重视及对市场动态的灵活应对。

管理人的因子库包含大约600个因子，其中基本面因子约200个，人工量价因子300余个，机器学习生成的单一因子以其独特的端到端处理能力成为重要组成部分，另类因子则聚焦于新闻舆情、供应链分析及宏观数据等维度。因子挖掘与处理方法上，基本面与人工量价因子依赖于传统的数据挖掘与分析，而机器学习因子则采用先进的算法从海量数据中提取非线性关系。所有因子通过线性组合形成预测模型，模型频段设定在3-5天，旨在捕捉短期至中期的市场趋势。

因子入库存储遵循严格的标准，尽管具体流程未详尽阐述，但强调了对因子逻辑性和有效性的持续评估，以及对因子表现不佳时的及时复审与优化机制。执行层面，管理人通过精选约2500只股票作为初始池，排除ST股、小市值、低流动性及低价股，实际操作中持有200至400只股票，与500指增和1000指增产品高度关联，换手率维持在60倍左右，借助第三方算法优化交易执行效率。风险控制方面，管理人实施严格的行业与风格偏离限制，同时计划逐步放宽行业约束以探索潜在的收益增厚机会。

对冲策略方面，管理人灵活运用多种工具，包括但不限于IM、融券及股指期货，经历了从使用远月合约转向

In [10]:
print(Ali_report)

管理人张润伟的量化策略综合了深度逻辑推理与先进的机器学习技术，旨在通过精密的基本面分析、量价信号的敏锐捕捉以及另类数据的独特洞察，构建出一套既稳健又灵活的投资框架。策略核心围绕着以基本面因子为主导，逐步融入并提升量价因子的有效性，特别是在国内市场环境下，量价因子的影响力显著，促使管理人在2022年10月引入了机器学习模型以进一步优化策略表现。截至2024年一季度，策略配置逻辑调整为基本面因子占比35%，人工挖掘的量价因子与机器学习量价因子分别占比35%和20%，另类因子占10%，体现出对多元化因子来源的重视及对市场动态的灵活应对。

管理人的因子库包含大约600个因子，其中基本面因子约200个，人工量价因子300余个，机器学习生成的单一因子以其独特的端到端处理能力成为重要组成部分，另类因子则聚焦于新闻舆情、供应链分析及宏观数据等维度。因子挖掘与处理方法上，基本面与人工量价因子依赖于传统的数据挖掘与分析，而机器学习因子则采用先进的算法从海量数据中提取非线性关系。所有因子通过线性组合形成预测模型，模型频段设定在3-5天，旨在捕捉短期至中期的市场趋势。

因子入库存储遵循严格的标准，尽管具体流程未详尽阐述，但强调了对因子逻辑性和有效性的持续评估，以及对因子表现不佳时的及时复审与优化机制。执行层面，管理人通过精选约2500只股票作为初始池，排除ST股、小市值、低流动性及低价股，实际操作中持有200至400只股票，与500指增和1000指增产品高度关联，换手率维持在60倍左右，借助第三方算法优化交易执行效率。风险控制方面，管理人实施严格的行业与风格偏离限制，同时计划逐步放宽行业约束以探索潜在的收益增厚机会。

对冲策略方面，管理人灵活运用多种工具，包括但不限于IM、融券及股指期货，经历了从使用远月合约转向近月合约的策略调整，以减少基差波动带来的风险。2023年对冲成本显著降低至4%，相比2022年的8%有明显改善，显示了管理人在成本控制上的进步。综上所述，管理人的量化策略不仅融合了传统与现代投资分析技术，还不断优化因子构成与模型结构，以适应不断变化的市场环境，同时在风险管理与交易执行上展现出高度的专业性和灵活性。


In [11]:
print(first_Ali_report)

管理人的量化策略整体融合了强逻辑分析与机器学习技术，旨在通过基本面分析、事件驱动策略与先进数据分析手段捕捉市场机遇。策略核心包括重视公司基本面数据（如盈利、营收）的逻辑分析，结合市场事件的响应，以及运用机器学习处理大数据，发现复杂市场模式。策略实施上，采取多策略融合以增强稳健性，并持续迭代模型以适应市场变化。因子组成方面，约一半收益源自基本面因子，如盈利质量与超预期动量，量价类与机器学习因子合计贡献约20%，另类因子虽未明确比例，但通过特定事件影响市场。因子挖掘与处理上，基本面与量价因子基于传统数据，机器学习因子则利用高级算法从多元数据中提炼，另类因子需人工加工特定事件数据。建模方法上，结合线性模型（如简单回归估算流动性）与机器学习模型（提高交易频率至半小时或一小时级别，可能应用监督学习的回归或分类算法预测市场动态）。因子入库存储条件未明确提及。执行层面，管理人通过自研IT系统支持策略快速迭代，持仓约50%日流动性股票，确保市场影响最小化，严格风控管理行业与风格偏离，控制Barra风格因子暴露，考虑非线性市值因子调整。关于年化双边换手率、具体持股数及对冲方法的详细数据未直接提供。


In [12]:
cost_list

[{'total_cost': 0.414076,
  'qwen-plus': {'cost': 0.414076,
   'prompt_tokens': 99235,
   'completion_tokens': 1428,
   'total_tokens': 100663}},
 {'total_cost': 0.24864,
  'qwen-max': {'cost': 0.24864,
   'prompt_tokens': 3594,
   'completion_tokens': 874,
   'total_tokens': 4468}}]

In [13]:
Quant_component_dic

{'quant_idea': ['管理人的量化策略整体上结合了强逻辑和机器学习的方法：\n\n1. **强逻辑和可解释性**：该公司重视基本面分析和事件驱动策略，这意味着他们在选股时会依据公司的基本面数据（如盈利、营收等）和市场事件（如公司公告、新闻等），这体现了策略中的强逻辑性和较高的可解释性。\n\n2. **机器学习的应用**：同时，公司也利用机器学习技术来处理大量数据和寻找市场中的复杂模式。这表明他们在策略中融入了较为先进的数据分析方法，以捕捉市场中的细微机会。\n\n3. **多策略融合**：公司采取多策略融合的方式，既包括了基于逻辑的基本面分析和事件驱动策略，也涵盖了利用机器学习进行的量化分析。这种方法有助于分散风险并提高策略的稳健性。\n\n4. **持续迭代和优化**：无论是基本面分析还是机器学习模型，公司都在不断地进行迭代和优化，以适应市场的变化并提高策略的有效性。\n\n综上所述，该公司的量化策略既注重策略的逻辑性和可解释性，同时也积极采用先进的机器学习技术，形成了一个多维度、灵活适应市场的投资体系。'],
 'factor_composition': ['根据文件内容，我们可以总结出以下关键信息：\n\n- 管理人的策略包含多个因子类别，主要包括量价类因子、基本面因子和另类因子。\n- 基本面因子在今年表现强劲，包括盈利质量、动量的超预期以及价值因子等。\n- 量价类因子也持续表现出较好的效果。\n- 另类因子贡献了部分收益，但触发时间不固定，依赖于特定事件的发生。\n\n具体占比如下：\n- 基本面因子：大约占总收益的一半。\n- 量价类因子和机器学习因子：合计占比约为20%多。\n- 另类因子：未给出具体比例，但从上下文推测其占比相对较小。\n\n因子总数没有明确给出，但从文中提到的不同因子类别及其细分来看，因子总数应该是由上述几类因子构成的，具体数量需要进一步分析策略细节才能确定。'],
 'factor_explanation': ['根据提供的内容，管理人讨论了不同类型的因子及其数据基础，主要包括：\n\n- **基本面因子**：这类因子在今年表现强劲，涉及盈利质量、盈利动量（超预期）、价值等。这些因子基于公司的财务报告数据，例如净利润、营业收入等。\n\n- **人工量价因子**：这部分因子来源于市场交易数据，包括量价关系等信息。\

In [ ]:
def report_updating(weak_config, strong_config,prompts,component_dict, requirements):
    